In [1]:
import os
import dask
from time import sleep
import utils.dirutils as dd
from dask.distributed import Client
import cupy as cp
import numpy as np
from dask_jobqueue import SLURMCluster
# import scipy as sp
# from numba import njit
import computation2_cluster as cc
import utils.dirutils as dd

/opt/conda/envs/RAPIDS/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
path = dd.base()
daskpath=os.path.join(path, "dask-worker-space-can-be-deleted")
try: 
    os.mkdir(daskpath) 
except OSError as error: 
    print(error)		 

# proc * threads = 18(cores)
# cluster = SLURMCluster(log_directory=daskpath) 
cluster = SLURMCluster(
    cores=12,
    queue='gpu',
    processes=3,
    log_directory=daskpath
)   

print(cluster.job_script())
cluster.scale(jobs=1) # 1 nodes
sleep(15)
client = Client(cluster)
client.amm.start()

[Errno 17] File exists: '/home/quojinhao/Data/dask-worker-space-can-be-deleted'


/opt/conda/envs/RAPIDS/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46797 instead
  warnings.warn(


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /home/quojinhao/Data/dask-worker-space-can-be-deleted/dask-worker-%J.err
#SBATCH -o /home/quojinhao/Data/dask-worker-space-can-be-deleted/dask-worker-%J.out
#SBATCH -p gpu
#SBATCH -n 1
#SBATCH --cpus-per-task=12
#SBATCH --mem=233G
#SBATCH -t 144:00:00

/opt/conda/envs/RAPIDS/bin/python -m distributed.cli.dask_worker tcp://192.168.0.254:43679 --nthreads 4 --nprocs 3 --memory-limit 77.61GiB --name dummy-name --nanny --death-timeout 90 --local-directory /scratch/$USER/$SLURM_JOB_ID --resources gpu=1 --protocol tcp://



In [3]:
client

<Client: 'tcp://192.168.0.254:43679' processes=3 threads=12, memory=232.83 GiB>

In [4]:
P1 = np.arange(1,3)
P2 = np.arange(3,5)
P3 = np.arange(5,7)
fileformat = "L{}widerbeam.h5"
nj = 100000
stepj = nj//10

In [5]:
def setup_gpu_worker(i):
    sleep(1.5)
    cp.cuda.Device(i).use()
    sleep(1.5)
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    setupGPUs = [ client.submit( setup_gpu_worker, i ) for i in range(3) ]

In [6]:
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    future1 = client.submit(cc.computation, P1, nj, stepj, fileformat)
    future2 = client.submit(cc.computation, P2, nj, stepj, fileformat)
    future3 = client.submit(cc.computation, P3, nj, stepj, fileformat)
# client.close()
# cluster.close()

In [ ]:
client.cancel(future1)
client.cancel(future2)
client.cancel(future3)

In [8]:
client.close()
cluster.close()

In [12]:
client.restart()

<Client: 'tcp://192.168.0.254:36169' processes=0 threads=0, memory=0 B>

In [ ]:
# cd ~/Data